In [ ]:
#-----------------------------------------------------------------------------
# START INIT ENVIRONMENT
#-----------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf /content/drive/Shareddrives/DA231-project/spark-3.0.3-bin-hadoop2.7.tgz

!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

Mounted at /content/drive


In [ ]:
#-----------------------------------------------------------------------------
# START OF PYSPARK INIT
#-----------------------------------------------------------------------------
import findspark

findspark.init()
findspark.find()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder\
         .master("local")\
         .appName("Colab")\
         .config('spark.ui.port', '4050')\
         .getOrCreate()
spark

In [ ]:
#-----------------------------------------------------------------------------
# Import packages
#-----------------------------------------------------------------------------
!pip install langdetect

from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import col,format_number
from pyspark.sql.functions import *
from langdetect import detect
from pyspark.sql.types import StringType
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
from pyspark.ml import Pipeline
import time
import datetime

# Install vader lexicon for sentiment analysis
!python -m nltk.downloader vader_lexicon
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
###!@2 END OF DEFINING INPUT FILES

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 26.4 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=3667cdb76210e4e12292058c5084909b80c4cf89a12c7ea79d93caa383d43f21
  Stored in directory: /root/.cache/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect
/usr/lib/python3.8/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
#-----------------------------------------------------------------------------
# User defined functions
#-----------------------------------------------------------------------------

def compute(y):
  desc = y[2]
  scores = sid.polarity_scores(desc)
  ans = list(scores.values())
  return y[0], y[1], y[2], ans[0], ans[1], ans[2], ans[3]

def sentiment(x):
  res = ""
  if x[1] >= x[2] and  x[1] >= x[3] and x[1] >= x[4]:
    res = "negative"
  elif x[2] >= x[1] and  x[2] >= x[3] and x[2] >= x[4]:
    res = "neutral"
  elif x[3] >= x[1] and  x[3] >= x[2] and x[3] >= x[4]:
    res = "positive"
  elif x[4] >= x[1] and  x[4] >= x[2] and x[4] >= x[3]:
    res = "compound"
  return x[0], x[1], x[2], x[3], x[4], res

# yyyy-mm-dd format
def parse_time(x):
  res1 = "null"
  res2 = "null"
  try:
    date_time = x[0].split()
    date = date_time[0].split('-')
    y = int(date[0])
    m = int(date[1])
    d = int(date[2])
    weekno = datetime.date(y, m, d).weekday()
    if weekno < 5:
      res1 = "Weekday"
    else:  # 5 Sat, 6 Sun
      res1 = "Weekend"

    time = date_time[1]
    hour = int(time.split(':')[0])
    if hour >= 0 and hour <6: # 0 - 5
      res2 = "mid-night"
    elif hour >= 6 and hour < 11: # 6 - 10
      res2 = "morning"
    elif hour >= 11 and hour < 16: # 11 - 15
      res2 = "afternoon"
    elif hour >= 16 and hour < 21: # 16 - 20
      res2 = "evening"
    elif hour >= 21 and hour <= 23: # 21 - 23
      res2 ="night"
    return x[0], x[1], x[2], res1,res2
  
  except:
    return 0, 0, 0, res1, res2

def parse_only_time(x):
    res = "null"
    try:
      time = x[0].split()[0]
      hour = int(time.split(':')[0].split('T')[1])
      if hour >= 0 and hour <6: # 0 - 5
        res = "mid-night"
      elif hour >= 6 and hour < 11: # 6 - 10
        res = "morning"
      elif hour >= 11 and hour < 16: # 11 - 15
        res = "afternoon"
      elif hour >= 16 and hour < 21: # 16 - 20
        res = "evening"
      elif hour >= 21 and hour <= 23: # 21 - 23
        res ="night"
      return x[0], hour, res
    except:
      return x[0], hour, res

def detect_language(text):
  try:
    return detect(text)
  except:
    return 'unknown'

def categorization_dict(filtered_list):
  word_count= len(filtered_list)
  pol_word= [x for x in filtered_list if x in polfi]
  ent_word= [x for x in filtered_list if x in ent]
  sp_word= [x for x in filtered_list if x in sp]
  cli_word= [x for x in filtered_list if x in cli]
  
  try:
    per_pol= len(pol_word)/word_count
  except:
    per_pol= 0
  
  try:
    per_ent= len(ent_word)/word_count
  except:
    per_ent=0

  try:
    per_sp= len(sp_word)/word_count
  except:
    per_sp=0
  

  try:
    per_cli= len(cli_word)/word_count
  except:
    per_cli=0

  category_dict= {
      "Politics,Business&Finance": per_pol,
       "Sports": per_sp, 
       "Environment&Climate": per_cli, 
       "Entertainment": per_ent
       }
  max_val = -1
  max_key = None
  for k,v in category_dict.items():
    if v > max_val:
      max_val = v
      max_key = k
  return max_key

In [ ]:
#-----------------------------------------------------------------------------
# Instagram Data - Create Path
#-----------------------------------------------------------------------------
path = "/content/drive/Shareddrives/DA231-project/instagram/"
insta_profiles = path + "instagram_profiles.csv"
insta_locations = path + "instagram_locations.csv"
insta_posts = path + "instagram_posts.csv"

In [ ]:
#-----------------------------------------------------------------------------
# Loading Instagram - Profiles data
#-----------------------------------------------------------------------------
df_insta_profiles = spark.read.option("header",True).option("delimiter", "\t").option("inferSchema",True).csv(insta_profiles)
df_insta_profiles.printSchema()

root
 |-- sid: string (nullable = true)
 |-- profile_id: string (nullable = true)
 |-- profile_name: string (nullable = true)
 |-- firstname_lastname: string (nullable = true)
 |-- description: string (nullable = true)
 |-- following: string (nullable = true)
 |-- followers: string (nullable = true)
 |-- n_posts: string (nullable = true)
 |-- url: string (nullable = true)
 |-- cts: string (nullable = true)
 |-- is_business_account: string (nullable = true)



In [ ]:
#-----------------------------------------------------------------------------
# Loading Instagram - Locations data
#-----------------------------------------------------------------------------
df_insta_locations = spark.read.option("header",True).option("delimiter", "\t").option("inferSchema",True).csv(insta_locations)
df_insta_locations.printSchema()

root
 |-- sid: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- street: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- city: string (nullable = true)
 |-- region: string (nullable = true)
 |-- cd: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- aj_exact_city_match: string (nullable = true)
 |-- aj_exact_country_match: string (nullable = true)
 |-- blurb: string (nullable = true)
 |-- dir_city_id: string (nullable = true)
 |-- dir_city_name: string (nullable = true)
 |-- dir_city_slug: string (nullable = true)
 |-- dir_country_id: string (nullable = true)
 |-- dir_country_name: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)
 |-- primary_alias_on_fb: string (nullable = true)
 |-- slug: string (nullable = true)
 |-- website: string (nullable = true)
 |-- cts: string (nullable = true)



In [ ]:
#-----------------------------------------------------------------------------
# Loading Instagram - Posts data
#-----------------------------------------------------------------------------
df_insta_posts = spark.read.option("header",True).option("delimiter", "\t").option("inferSchema",True).csv(insta_posts)
df_insta_posts.printSchema()

root
 |-- sid: string (nullable = true)
 |-- sid_profile: string (nullable = true)
 |-- post_id: string (nullable = true)
 |-- profile_id: string (nullable = true)
 |-- location_id: string (nullable = true)
 |-- cts: string (nullable = true)
 |-- post_type: string (nullable = true)
 |-- description: string (nullable = true)
 |-- numbr_likes: string (nullable = true)
 |-- number_comments: string (nullable = true)



In [ ]:
#-----------------------------------------------------------------------------
# Cache Instagram data for performance benefits
#-----------------------------------------------------------------------------
df_insta_profiles.cache()
print(f'\nInsta Profiles record count : {df_insta_profiles.count()}')
print(f'\nInsta Profiles sample data \n')
df_insta_profiles.show()

#-----------------------------------------------------------------------------
df_insta_locations.cache()
print(f'\nInsta Locations record count : {df_insta_locations.count()}')
print(f'\nInsta Locations sample data \n')
df_insta_locations.show()

#-----------------------------------------------------------------------------
df_insta_posts.cache()
print(f'\nInsta Posts record count : {df_insta_posts.count()}')
print(f'\nInsta Posts sample data \n')
df_insta_posts.show()

#-----------------------------------------------------------------------------


Insta Profiles record count : 4509879

Insta Profiles sample data 

+-------+----------+-----------------+--------------------+--------------------+---------+---------+-------+--------------------+--------------------+-------------------+
|    sid|profile_id|     profile_name|  firstname_lastname|         description|following|followers|n_posts|                 url|                 cts|is_business_account|
+-------+----------+-----------------+--------------------+--------------------+---------+---------+-------+--------------------+--------------------+-------------------+
|4184446|4721049632|      jphillip033|         John Pierce|"""Document Every...|      250|      146|    170|                null|2019-08-01 14:38:...|              false|
|4184457| 590583002|        mama_haas|              Deanna|Trying to enjoy t...|      534|     1145|   2878|www.etsy.com/shop...|2019-08-01 14:39:...|              false|
|4184460|1074146658| yellowlightbreen|          Eliza Gray| Maine is for Lov

In [ ]:
#-----------------------------------------------------------------------------
# Loading Youtube data - Add COUNTRY column to corresponding files
#-----------------------------------------------------------------------------
path = "/content/drive/Shareddrives/DA231-project/youtube/"

youtube_US = path + "US_youtube_trending_data.csv"
youtube_RU = path + "RU_youtube_trending_data.csv"
youtube_MX = path + "MX_youtube_trending_data.csv"
youtube_KR = path + "KR_youtube_trending_data.csv"
youtube_JP = path + "JP_youtube_trending_data.csv"
youtube_IN = path + "IN_youtube_trending_data.csv"
youtube_GB = path + "GB_youtube_trending_data.csv"
youtube_FR = path + "FR_youtube_trending_data.csv"
youtube_DE = path + "DE_youtube_trending_data.csv"
youtube_CA = path + "CA_youtube_trending_data.csv"
youtube_BR = path + "BR_youtube_trending_data.csv"

spark.read.option("header",True).option("inferSchema",True).csv(youtube_US).createOrReplaceTempView('t_youtube_US')
spark.read.option("header",True).option("inferSchema",True).csv(youtube_RU).createOrReplaceTempView('t_youtube_RU')
spark.read.option("header",True).option("inferSchema",True).csv(youtube_MX).createOrReplaceTempView('t_youtube_MX')
spark.read.option("header",True).option("inferSchema",True).csv(youtube_KR).createOrReplaceTempView('t_youtube_KR')
spark.read.option("header",True).option("inferSchema",True).csv(youtube_JP).createOrReplaceTempView('t_youtube_JP')
spark.read.option("header",True).option("inferSchema",True).csv(youtube_IN).createOrReplaceTempView('t_youtube_IN')
spark.read.option("header",True).option("inferSchema",True).csv(youtube_GB).createOrReplaceTempView('t_youtube_GB')
spark.read.option("header",True).option("inferSchema",True).csv(youtube_FR).createOrReplaceTempView('t_youtube_FR')
spark.read.option("header",True).option("inferSchema",True).csv(youtube_DE).createOrReplaceTempView('t_youtube_DE')
spark.read.option("header",True).option("inferSchema",True).csv(youtube_CA).createOrReplaceTempView('t_youtube_CA')
spark.read.option("header",True).option("inferSchema",True).csv(youtube_BR).createOrReplaceTempView('t_youtube_BR')



df_youtube = spark.sql('select \'US\' as country, * from t_youtube_US union \
                        select \'RU\' as country, * from t_youtube_RU union \
                        select \'MX\' as country, * from t_youtube_MX union \
                        select \'KR\' as country, * from t_youtube_KR union \
                        select \'JP\' as country, * from t_youtube_JP union \
                        select \'IN\' as country, * from t_youtube_IN union \
                        select \'GB\' as country, * from t_youtube_GB union \
                        select \'FR\' as country, * from t_youtube_FR union \
                        select \'DE\' as country, * from t_youtube_DE union \
                        select \'CA\' as country, * from t_youtube_CA union \
                        select \'BR\' as country, * from t_youtube_BR')

#-----------------------------------------------------------------------------
df_youtube.cache()
print(f'\nYoutube record count : {df_youtube.count()}')
print(f'\nYoutube sample data \n')
df_youtube.show(20)



Youtube record count : 1890577

Youtube sample data 

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+----------+-------+--------+-------------+--------------------+-----------------+----------------+--------------------+
|country|            video_id|               title|         publishedAt|           channelId|        channelTitle|categoryId|       trending_date|                tags|view_count|  likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|         description|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+----------+-------+--------+-------------+--------------------+-----------------+----------------+--------------------+
|     US|         5V2B28OqfqM|I Spent $1,000,00...|2020-08-08T20:00:02Z|UCX6OQ3Dkcsb

In [ ]:
#-----------------------------------------------------------------------------
# Create Dataframe with Youtube Category ID and Name Mappings
#-----------------------------------------------------------------------------
data = [{'category_id': 1, 'category_name' : 'Film & Animation'},
        {'category_id': 2, 'category_name' : 'Autos & Vehicles'},
        {'category_id': 3, 'category_name' : 'Not available'},
        {'category_id': 4, 'category_name' : 'Not available'},
        {'category_id': 5, 'category_name' : 'Not available'},
        {'category_id': 5, 'category_name' : 'Not available'},
        {'category_id': 6, 'category_name' : 'Not available'},
        {'category_id': 7, 'category_name' : 'Not available'},
        {'category_id': 8, 'category_name' : 'Not available'},
        {'category_id': 9, 'category_name' : 'Not available'},
        {'category_id': 10, 'category_name' : 'Music'},
        {'category_id': 11, 'category_name' : 'Not available'},
        {'category_id': 12, 'category_name' : 'Not available'},
        {'category_id': 13, 'category_name' : 'Not available'},
        {'category_id': 14, 'category_name' : 'Not available'},
        {'category_id': 15, 'category_name' : 'Pets & Animals'},
        {'category_id': 16, 'category_name' : 'Not available'},
        {'category_id': 17, 'category_name' : 'Sports'},
        {'category_id': 18, 'category_name' : 'Short Movies'},
        {'category_id': 19, 'category_name' : 'Travel & Events'},
        {'category_id': 20, 'category_name' : 'Gaming'},
        {'category_id': 21, 'category_name' : 'Videoblogging'},
        {'category_id': 22, 'category_name' : 'People & Blogs'},
        {'category_id': 23, 'category_name' : 'Comedy'},
        {'category_id': 24, 'category_name' : 'Entertainment'},
        {'category_id': 25, 'category_name' : 'News & Politics'},
        {'category_id': 26, 'category_name' : 'Howto & Style'},
        {'category_id': 27, 'category_name' : 'Education'},
        {'category_id': 28, 'category_name' : 'Science & Technology'},
        {'category_id': 29, 'category_name' : 'Nonprofits & Activism'},
        {'category_id': 30, 'category_name' : 'Movies'},
        {'category_id': 31, 'category_name' : 'Animation'},
        {'category_id': 32, 'category_name' : 'Action/Adventure'},
        {'category_id': 33, 'category_name' : 'Classics'},
        {'category_id': 34, 'category_name' : 'Comedy'},
        {'category_id': 35, 'category_name' : 'Documentary'},
        {'category_id': 36, 'category_name' : 'Drama'},
        {'category_id': 37, 'category_name' : 'Family'},
        {'category_id': 38, 'category_name' : 'Foreign'},
        {'category_id': 39, 'category_name' : 'Horror'},
        {'category_id': 40, 'category_name' : 'Sci-Fi/Fantasy'},
        {'category_id': 41, 'category_name' : 'Thriller'},
        {'category_id': 42, 'category_name' : 'Shorts'},
        {'category_id': 43, 'category_name' : 'Shows'},
        {'category_id': 44, 'category_name' : 'Trailers'}
        ]


df_youtube_category_mapping = spark.createDataFrame(data)

df_youtube_category_mapping.show(50)

/content/spark-3.0.3-bin-hadoop2.7/python/pyspark/sql/session.py:381: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-----------+--------------------+
|category_id|       category_name|
+-----------+--------------------+
|          1|    Film & Animation|
|          2|    Autos & Vehicles|
|          3|       Not available|
|          4|       Not available|
|          5|       Not available|
|          5|       Not available|
|          6|       Not available|
|          7|       Not available|
|          8|       Not available|
|          9|       Not available|
|         10|               Music|
|         11|       Not available|
|         12|       Not available|
|         13|       Not available|
|         14|       Not available|
|         15|      Pets & Animals|
|         16|       Not available|
|         17|              Sports|
|         18|        Short Movies|
|         19|     Travel & Events|
|         20|              Gaming|
|         21|       Videoblogging|
|         22|      People & Blogs|
|         23|              Comedy|
|         24|       Entertainment|
|         25|     Ne

In [ ]:
#-----------------------------------------------------------------------------
# Create Youtube Master Data
#-----------------------------------------------------------------------------
df_youtube.createOrReplaceTempView('t_youtube')
df_youtube_category_mapping.createOrReplaceTempView('t_youtube_category_mapping')

df_youtube_master_data = spark.sql('select distinct country, video_id, title as video_title, publishedAt, channelId, channelTitle, categoryId, category_name, view_count, likes, dislikes, comment_count, trending_date \
          from t_youtube join t_youtube_category_mapping on categoryId = category_id')

#df_youtube_master_data.explain(mode='formatted')

#-----------------------------------------------------------------------------
df_youtube_master_data.cache()
print(f'\nYoutube Master Data record count : {df_youtube_master_data.count()}')
print(f'\nYoutube Master sample data \n')
df_youtube_master_data.show(20)

#-----------------------------------------------------------------------------
df_youtube_master_data.createOrReplaceTempView('t_youtube_master_data')


Youtube Master Data record count : 1834966

Youtube Master sample data 

+-------+-----------+-------------------------------------+--------------------+--------------------+----------------------------+----------+-------------+----------+------+--------+-------------+--------------------+
|country|   video_id|                          video_title|         publishedAt|           channelId|                channelTitle|categoryId|category_name|view_count| likes|dislikes|comment_count|       trending_date|
+-------+-----------+-------------------------------------+--------------------+--------------------+----------------------------+----------+-------------+----------+------+--------+-------------+--------------------+
|     US|DFjkHnZCqmU|                 Last Minute Super...|2022-02-09T15:00:13Z|UChBEbMKI1eCcejTt...|             Joshua Weissman|        26|Howto & Style|    584408| 25936|       0|          887|2022-02-10T00:00:00Z|
|     FR|HTD8-Mps__g|                 Cendre de bois a

In [ ]:
# Range of dataset 

dataframe = df_insta_profiles.filter(~col("cts").contains("7275"))
min_date, max_date = dataframe.select("cts").selectExpr("CAST(cts AS date) AS date").select(min("date"), max("date")).first()
print("Range -> Instagram Profiles")
print("min_date = ",min_date)
print("max_date = ",max_date)
print()
print("Range -> Instagram Locations")
min_date, max_date = df_insta_locations.select("cts").selectExpr("CAST(cts AS date) AS date").select(min("date"), max("date")).first()
print("min_date = ",min_date)
print("max_date = ",max_date)
print()
print("Range -> Instagram Posts")
min_date, max_date = df_insta_posts.select("cts").selectExpr("CAST(cts AS date) AS date").select(min("date"), max("date")).first()
print("min_date = ",min_date)
print("max_date = ",max_date)
print()
print("Range -> Youtube")
min_date, max_date = df_youtube_master_data.select("publishedAt").selectExpr("CAST(publishedAt AS date) AS date").select(min("date"), max("date")).first()
print("min_date = ",min_date)
print("max_date = ",max_date)
print()

Range -> Instagram Profiles
min_date =  2018-05-25
max_date =  2019-08-19

Range -> Instagram Locations
min_date =  2019-02-26
max_date =  2019-06-23

Range -> Instagram Posts
min_date =  2010-07-19
max_date =  2019-08-30

Range -> Youtube
min_date =  2020-07-14
max_date =  2022-11-27



In [ ]:
#-----------------------------------------------------------------------------
# YouTube :: Country => Top Categories => Top Channels
#-----------------------------------------------------------------------------

start = time.time()

# Assigning weights and calculating popularity score at video level 
spark.sql('select \
               country \
              ,video_id \
              ,video_title \
              ,channelTitle \
              ,publishedAt \
              ,category_name \
              ,sum(view_count * 0.6 + likes * 0.15 + dislikes * 0.15 + comment_count * 0.10) as popularity_score \
            from \
              t_youtube_master_data \
            group by \
               country \
              ,video_id \
              ,video_title \
              ,channelTitle \
              ,publishedAt \
              ,category_name \
              ,view_count, likes, dislikes, comment_count').createOrReplaceTempView('pop_score_calc')

# Aggregate popularity scores at Country and Category Level
spark.sql('select                                              \
              country, video_title, channelTitle, publishedAt, category_name, \
              round(sum(popularity_score) over (partition by country,category_name) /100000,0) as country_category_popularity_score, \
              round(sum(popularity_score) over (partition by country,category_name,channelTitle) /10000,0) as country_category_channel_popularity_score \
            from \
              pop_score_calc').createOrReplaceTempView('pop_score_sum')

# Rank based on popularity scores
pop_rank = spark.sql('select \
              country, video_title, video_title, channelTitle, publishedAt, category_name, \
              country_category_popularity_score, \
              dense_rank() over (partition by country order by country_category_popularity_score desc) as country_category_popularity_rank, \
              country_category_channel_popularity_score, \
              dense_rank () over (partition by country, category_name order by country_category_channel_popularity_score desc) as country_category_channel_popularity_rank\
            from \
              pop_score_sum ')

# Caching data and get record count
pop_rank.cache()
print(f'\nYoutube Popularity Ranking Data record count : {pop_rank.count()}')


pop_rank.createOrReplaceTempView('pop_rank')

df_youtube_popularity_analysis_output = spark.sql("select \
              country, category_name, \
              country_category_popularity_rank, \
              channelTitle, \
              country_category_channel_popularity_rank \
            from \
              pop_rank \
            where \
              country_category_popularity_rank <= 10 and country_category_channel_popularity_rank <= 10 \
            group by \
              country, category_name, \
              country_category_popularity_rank, \
              channelTitle, \
              country_category_channel_popularity_rank \
            order by \
              country,\
              country_category_popularity_rank asc, country_category_channel_popularity_rank asc")

df_youtube_popularity_analysis_output.show(1500)

end = time.time()
print(f'Execution time {(end - start)} seconds')


Youtube Popularity Ranking Data record count : 1834686
+-------+--------------------+--------------------------------+------------------------------------+----------------------------------------+
|country|       category_name|country_category_popularity_rank|                        channelTitle|country_category_channel_popularity_rank|
+-------+--------------------+--------------------------------+------------------------------------+----------------------------------------+
|     BR|               Music|                               1|                           BLACKPINK|                                       1|
|     BR|               Music|                               1|                         HYBE LABELS|                                       2|
|     BR|               Music|                               1|                              SMTOWN|                                       3|
|     BR|               Music|                               1|                   JYP Entert

In [ ]:
#-----------------------------------------------------------------------------
# YouTube :: What are popular days and months to publish videos in each category? 
# 1: Sunday ; 7: Saturday
#-----------------------------------------------------------------------------

start = time.time()

#-----------------------------------------------------------------------------
# Day analysis
#-----------------------------------------------------------------------------
spark.sql("select category_name, dayofweek(publishedAt) as day_of_week , \
            count(*) as day_of_week_video_published_count \
           from pop_rank \
           where country_category_channel_popularity_rank <= 10 \
           group by category_name, dayofweek(publishedAt) \
           order by 1, 3 desc ").createOrReplaceTempView('category_publish_dates_count')

spark.sql("select \
            category_name, day_of_week, \
            round((day_of_week_video_published_count / (sum(day_of_week_video_published_count) over (partition by category_name)) * 100),2) as percent_videos_published \
           from category_publish_dates_count \
           order by 1, 3 desc").show(500)

#-----------------------------------------------------------------------------
# Monthly analysis
#-----------------------------------------------------------------------------
spark.sql("select category_name, month(publishedAt) as published_month , \
            count(*) as published_month_video_count \
           from pop_rank \
           where country_category_channel_popularity_rank <= 10 \
           group by category_name, month(publishedAt) \
           order by 1, 3 desc ").createOrReplaceTempView('category_publish_month_count')

spark.sql("select \
            category_name, published_month, \
            round((published_month_video_count / (sum(published_month_video_count) over (partition by category_name)) * 100),2) as percent_videos_published \
           from category_publish_month_count \
           order by 1, 3 desc").show(500)

#-----------------------------------------------------------------------------
# Hourly analysis
#-----------------------------------------------------------------------------
spark.sql("select category_name, hour(publishedAt) as published_hour , \
            count(*) as published_hour_video_count \
           from pop_rank \
           where country_category_channel_popularity_rank <= 10 \
           group by category_name, hour(publishedAt) \
           order by 1, 3 desc ").createOrReplaceTempView('category_publish_hour_count')

spark.sql("select \
            category_name, published_hour, \
            round((published_hour_video_count / (sum(published_hour_video_count) over (partition by category_name)) * 100),2) as percent_videos_published \
           from category_publish_hour_count \
           order by 1, 3 desc").show(500)


end = time.time()
print(f'Execution time {(end - start)} seconds')

+--------------------+-----------+------------------------+
|       category_name|day_of_week|percent_videos_published|
+--------------------+-----------+------------------------+
|    Autos & Vehicles|          5|                   18.41|
|    Autos & Vehicles|          4|                   16.77|
|    Autos & Vehicles|          2|                   15.27|
|    Autos & Vehicles|          6|                   14.14|
|    Autos & Vehicles|          3|                    14.1|
|    Autos & Vehicles|          1|                   12.65|
|    Autos & Vehicles|          7|                    8.67|
|              Comedy|          1|                   16.71|
|              Comedy|          4|                   16.28|
|              Comedy|          7|                    15.6|
|              Comedy|          5|                   14.48|
|              Comedy|          6|                   13.96|
|              Comedy|          3|                   11.93|
|              Comedy|          2|      

In [ ]:
#pop_rank.unpersist()

df_youtube_popularity_analysis_output.explain(mode = 'formatted')

In [ ]:
#-----------------------------------------------------------------------------
# Instagram :: Which were the popular locations used for the posts?
#-----------------------------------------------------------------------------

df_insta_locations.createOrReplaceTempView('q3t_insta_locations')
df_insta_posts.createOrReplaceTempView('q3t_insta_posts')
spark.sql('select location_id, COUNT(*) as count from q3t_insta_posts group by location_id').createOrReplaceTempView('q3t_locations_count')
spark.sql('select location_id, city, count from q3t_locations_count join q3t_insta_locations on location_id = id order by 3 desc').show(50, truncate=False)

+-----------+--------------------------------------------+------+
|location_id|city                                        |count |
+-----------+--------------------------------------------+------+
|212988663  |New York, New York                          |157368|
|213385402  |London, United Kingdom                      |111762|
|17326249   |Moscow, Russia                              |75006 |
|212999109  |Los Angeles, California                     |74384 |
|213174824  |Saint Petersburg, Russia                    |48501 |
|212950988  |Brooklyn, New York                          |41387 |
|6889842    |Paris, France                               |36751 |
|359545221  |Moscow, Russia                              |33389 |
|20188833   |Manhattan, New York                         |29031 |
|204517928  |Chicago, Illinois                           |25400 |
|212991559  |Toronto, Ontario                            |24719 |
|44961364   |San Francisco, California                   |19900 |
|212941492

In [ ]:
#-----------------------------------------------------------------------------
# Instagram :: what is the sentiment of posts (neg , pos, neutral)? 
#-----------------------------------------------------------------------------

df_insta_posts_sub = df_insta_posts.na.drop(subset=["location_id", "description"]).sample(0.05, 3) # considering only a portion of insta posts dataset due to too many number of data points (~4 CR)
df_insta_posts_sub.createOrReplaceTempView('q4t_insta_posts')
df_insta_locations.createOrReplaceTempView('q4t_insta_locations')
df_reduced = spark.sql('select location_id, city, description from q4t_insta_locations join q4t_insta_posts on location_id = id')

rdd41 = df_reduced.rdd.map(compute)
dfq41 = rdd41.toDF(["location_id","city","description","neg","neu","pos","compound"])
dfq41.createOrReplaceTempView('q4t_insta_posts_sentiment')

df42 = spark.sql('select city, round(avg(neg),2) as avg_neg, round(avg(neu),2) as avg_neu, round(avg(pos),2) as avg_pos, round(avg(compound),2) as avg_compound from q4t_insta_posts_sentiment group by city')
rdd42 = df42.rdd.map(sentiment)
out_df4 = rdd42.toDF(["city","avg_neg","avg_neu","avg_pos","avg_compound","overall"])

out_df4.where(col("overall") == "negative").show(truncate=False)
out_df4.where(col("overall") == "neutral").show(truncate=False)
out_df4.where(col("overall") == "positive").show(truncate=False)
out_df4.where(col("overall") == "compound").show(truncate=False)

+-------------------------------------------------+-------+-------+-------+------------+--------+
|city                                             |avg_neg|avg_neu|avg_pos|avg_compound|overall |
+-------------------------------------------------+-------+-------+-------+------------+--------+
|Vinterviken, Stockholms Län, Sweden              |0.0    |0.0    |0.0    |0.0         |negative|
|Spasskolutovinovskoye, Orlovskaya Oblast', Russia|0.0    |0.0    |0.0    |0.0         |negative|
|Zederhaus                                        |0.0    |0.0    |0.0    |0.0         |negative|
|Aughrim, Wicklow                                 |0.78   |0.22   |0.0    |-0.56       |negative|
|Kotel, Bulgaria                                  |0.0    |0.0    |0.0    |0.0         |negative|
|Paga, Nusa Tenggara Timur, Indonesia             |0.0    |0.0    |0.0    |0.0         |negative|
|La Joya, Baja California, Mexico                 |0.0    |0.0    |0.0    |0.0         |negative|
|Saint-Jules        

In [ ]:
#-----------------------------------------------------------------------------
# Instagram :: 
# What is the day/time range at which people like to post? 
# a.  Days- Weekend (sat-sun) or weekday (mon-fri)
# b.  Time- 
#     i.    Mid-night: 0 - 5
#     ii.   Morning: 6 -10
#     iii.  Afternoon: 11 - 15
#     iv.   Evening: 16 - 20
#     v.    Night: 21 - 23
#-----------------------------------------------------------------------------

time_df = spark.sql('select cts, numbr_likes, number_comments from q4t_insta_posts')

q5t_rdd = time_df.rdd.map(parse_time)
q5df_t = q5t_rdd.toDF(["cts","numbr_likes","number_comments","day","time"])
q5df_t = q5df_t.na.drop(subset=["day", "time"])
q5df_t.createOrReplaceTempView('q5t_insta_posts_time_analysis')
spark.sql('select day, time, numbr_likes, number_comments from q5t_insta_posts_time_analysis where time != "null" and day != "null"').createOrReplaceTempView('q5t_insta_time_analysis')

# Days
df51_out = spark.sql('select day, sum(numbr_likes) as total_likes, sum(number_comments) as total_comments from q5t_insta_time_analysis group by day')
df51_out.select("day", format_number(col("total_likes"),0).alias("total_likes"),format_number(col("total_comments"),0).alias("total_comments")).show()
# Time
df52_out = spark.sql('select time, sum(numbr_likes) as total_likes, sum(number_comments) as total_comments from q5t_insta_time_analysis group by time')
df52_out.select("time", format_number(col("total_likes"),0).alias("total_likes"),format_number(col("total_comments"),0).alias("total_comments")).show()

+-------+-----------+--------------+
|    day|total_likes|total_comments|
+-------+-----------+--------------+
|Weekday|296,982,484|     8,051,431|
|Weekend|128,107,627|     3,169,015|
+-------+-----------+--------------+

+---------+-----------+--------------+
|     time|total_likes|total_comments|
+---------+-----------+--------------+
|afternoon|101,392,614|     2,592,325|
|    night| 62,029,803|     1,561,481|
|  morning| 58,581,266|     1,565,180|
|  evening|129,523,901|     3,473,958|
|mid-night| 73,562,527|     2,027,502|
+---------+-----------+--------------+



In [ ]:
#-----------------------------------------------------------------------------
# YouTube :: 
# In which time duration, there were most number of videos published?
# i. Mid-night: 0 - 5
# ii. Morning: 6 -10
# iii. Afternoon: 11 - 15
# iv.Evening: 16 - 20
# v. Night: 21 - 23
#-----------------------------------------------------------------------------

dfq6 = spark.sql('select publishedAt from t_youtube_master_data')
rdd6 = dfq6.rdd.map(parse_only_time)
dfq6_t = rdd6.toDF(["publishedAt", "hour", "time"])
dfq6_t = dfq6_t.na.drop(subset=["publishedAt"])
dfq6_t.createOrReplaceTempView('q6t_youtube_time_analysis')

spark.sql('select * from q6t_youtube_time_analysis where time != "null"').createOrReplaceTempView('q6t_youtube_time_analysis')
out_df6 = spark.sql('select time, count(hour) as total_videos from q6t_youtube_time_analysis group by time')
out_df6.select("time", format_number(col("total_videos"),0).alias("total_videos")).show()

+---------+------------+
|     time|total_videos|
+---------+------------+
|afternoon|     523,090|
|    night|     208,083|
|  morning|     363,511|
|  evening|     465,154|
|mid-night|     274,848|
+---------+------------+



In [ ]:
#-----------------------------------------------------------------------------
# Instagram :: 
# which channel has the highest view count? list top 10 video in decreasing order of #view-count
# which channel has the highest likes? list top 10 video in decreasing order of #likes
# Approximate Revenue for the top 10 videos
# Duration of the video to trend
#-----------------------------------------------------------------------------

from pyspark.sql import functions as F
from pyspark.sql.functions import create_map, lit
from itertools import chain
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import round

catergory_based_on_id = {
	'1': 'Film & Animation',
	'2': 'Autos & Vehicles',
	'10': '	Music',
	'15': 'Pets & Animals',
	'17': 'Sports',
	'18': 'Short Movies',
	'19': 'Travel & Events',
	'20': 'Gaming',
	'21': 'Videoblogging',
	'22': 'People & Blogs',
	'24': 'Entertainment',
	'26': 'Howto & Style',
	'27': 'Education',
	'28': 'Science & Technology',
	'29': 'Nonprofits & Activism',
	'30': 'Movies',
	'31': 'Anime/Animation',
	'32	': 'Action/Adventure',
	'33': 'Classics',
	'34': 'Comedy',
	'35': 'Documentary',
	'36': 'Drama',
	'37': 'Family',
	'38': 'Foreign',
	'39': 'Horror',
	'40': 'Sci-Fi/Fantasy',
	'41': 'Thriller',
	'42': 'Shorts',
	'43': 'Shows',
	'44': 'Trailers',
	'25': 'News',
	'23': 'Random'
}

distinctDF = df_youtube_master_data.distinct()

mapping = create_map([lit(x) for x in chain(*catergory_based_on_id.items())])



clean_data = distinctDF.dropDuplicates(['video_title']).withColumn("view_count",col("view_count").cast("int")).withColumn("likes",col("likes").cast("int")).na.drop().select(['video_title', 'view_count', 'likes', 'channelTitle', 'publishedAt', 'trending_date', 'categoryId'])


report = clean_data.select(['video_title', 'view_count', 'likes', 'channelTitle', 'publishedAt', 'trending_date', 'categoryId']).sort(['view_count', 'likes'], ascending=False).limit(1000)
test = report.withColumn('publishedAt',to_timestamp(col('publishedAt'))) \
  .withColumn('trending_date',to_timestamp(col('trending_date'))) \
  .withColumn('category',col('categoryId')) \
  .withColumn('DiffInSeconds',col("trending_date").cast("long") - col('publishedAt').cast("long")) \
  .withColumn("DiffInDaysToTrend",round(col("DiffInSeconds")/86400)).withColumn("ApproximateRevenueInDollors",round(col("view_count")/1000*4)).na.replace(catergory_based_on_id, 1).drop("DiffInSeconds","categoryId")

print()
test.show(truncate=False)

tot = test.count()

test.groupBy("category") \
  .count() \
  .withColumnRenamed('count', 'total_count') \
  .withColumn('Earning_Potential', (F.col('total_count') / tot) * 100 ) \
  .show()


+-----------------------------------------------------------+----------+--------+-------------------+-------------------+-------------------+--------------+-----------------+---------------------------+
|video_title                                                |view_count|likes   |channelTitle       |publishedAt        |trending_date      |category      |DiffInDaysToTrend|ApproximateRevenueInDollors|
+-----------------------------------------------------------+----------+--------+-------------------+-------------------+-------------------+--------------+-----------------+---------------------------+
|BTS (방탄소년단) 'Butter' Official MV                      |278080610 |16213755|HYBE LABELS        |2021-05-21 03:46:13|2021-06-01 00:00:00|	Music        |11.0             |1112322.0                  |
|BLACKPINK - ‘Pink Venom’ M/V                               |229948883 |12347429|BLACKPINK          |2022-08-19 04:00:13|2022-08-31 00:00:00|	Music        |12.0             |919796.0          

In [ ]:
#-----------------------------------------------------------------------------
# Instagram :: which users have the highest followers (#likes’, ‘#comments’ and ‘#followers’)? list top 10 users in decreasing order
#-----------------------------------------------------------------------------

block1 = df_insta_profiles.withColumn("followers",col("followers").cast("int")).withColumn("n_posts",col("n_posts").cast("int")).na.drop().select(['profile_name', 'profile_id', 'followers', 'n_posts'])
block2 = df_insta_posts.withColumn("numbr_likes",col("numbr_likes").cast("int")).withColumn("number_comments",col("number_comments").cast("int")).na.drop().select(['profile_id', 'numbr_likes', 'number_comments', 'cts','description'])
block3 = block1.join(block2, [block1.profile_id == block2.profile_id] , how = 'inner' )

df_insta_posts_sub = block3.na.drop(subset=["profile_name", "followers", "n_posts", "numbr_likes", "number_comments", "cts", "description"]).sample(0.02, 3)
block4 = df_insta_posts_sub.select(['profile_name', block2.profile_id,  'followers', 'n_posts']).distinct()
block4.sort(['followers', 'numbr_likes', 'number_comments', 'n_posts'], ascending=False).show(10)

+-------------+----------+---------+-------+
| profile_name|profile_id|followers|n_posts|
+-------------+----------+---------+-------+
|     lunamaya|   1948416| 21513685|   3136|
|    lizakoshy|   8443283| 17920459|   1782|
|  ussypratama|   9902057| 11672975|   6884|
|    aurakasih|   5451786|  9400152|   2025|
|  audimarissa|  12830386|  9369877|    995|
|    tatizaqui| 223167162|  8843850|   6052|
|   nahcardoso|   8190340|  7869601|   6551|
|afnan_albatel|  55336452|  7757158|   9284|
|  cosculluela|  39901137|  7224572|   2805|
|  cyrineanour|1588784435|  6498207|   1221|
+-------------+----------+---------+-------+
only showing top 10 rows



In [ ]:
#-----------------------------------------------------------------------------
# Instagram :: Which are the Popular Instagram tags and its occurances
#-----------------------------------------------------------------------------

df_insta_posts_sub.createOrReplaceTempView("post_tempview")
post_df = spark.sql("""select  trim(trailing ',' from regexp_replace(description, '.*?(#\\\\w+)|.*', '$1,')) as Hastags 
  from    post_tempview
  where   description like '%#%'
""")

final_df = post_df.withColumn('PopularHastags', F.explode(F.split(F.col('Hastags'), ',')))\
    .groupBy('PopularHastags')\
    .count()\
    .sort('count', ascending=False).na.drop(subset=["PopularHastags"])\
    .limit(10)
final_df.filter(F.col('PopularHastags').contains('#')).show()

+--------------+-----+
|PopularHastags|count|
+--------------+-----+
|         #love| 2091|
|       #travel| 1415|
|    #instagood| 1305|
|  #photography| 1280|
|      #fashion| 1211|
|       #beauty| 1173|
|#photooftheday| 1075|
|          #art|  980|
|       #london|  895|
+--------------+-----+



In [ ]:
#-----------------------------------------------------------------------------
# Instagram :: Categorization of posts using NLP (/text-analytics) on ‘description’ feature a. Political b. Sports c. Entertainment d. Business and finance e. Environment and climate
#-----------------------------------------------------------------------------

polfi= ['politics','world record','guinness','debate','war','religion','payoff','income','partnership','liable','loan','debt','profit','government','govern','congress','trump','biden','modi','independence','freedom','speech','modi','modiji','racist','secularism','secular','liberal','promises','india','indian government','congressmen','bjp','aap','rjd','state','public','property','capital','stock','prices','price','national','act','article','section','people','education','security','work','sector','farmers','jobs','agriculture','jobs','salary','business','industry','worker','labour','manufacturing','reservation','caste','country','news','judiciary','legislative','council','members','capitalism','earning','expenditure','spend','budget','account','business plan','currency','market','sensex','nifty','cost','assets','interest','inventory','interest rate','bank','rbi','isis','terrorism','terror','flight','ipo','crime','hijack','rape','rapist','jail','kill','invest','money','company','companies','industry','employee','firm','firing','recession','police','republic','republican','donation','ceo','turnover','lay off','economy','funds','deal','reliance','tata']
sp= list("IndVSPak,ball,chess, World cup, ipl, virat kohli, msd, dhoni, Toss, Run, Wicket, Pitch, Stump, Bails, Crease, Pavilion, Gloves, Wicket Keeper, Over, Followon, Rubber, Spin, Ashes, Catch, Bowled, Stump out, Runout, L. B. W; Hit Wicket, Googley, Not out, No ball, Wide ball, Dead ball, Maiden over, Overthrow, Bye, Leg by, Cover drive, Late cut, Hook, Glance, Stroke, Shot, Pull, Sixer, Follow Through, Turn, Bouncer, Hattrick, Round the wicket, Over the wicket, Seamer, Boundry line, Slip, Square leg, Runner, Cover, Yorker, Gully, Long on, Silly point, Midwicket, Mid on, Forward short leg,Knock out, Ring Stoppage, Punch, Round, Upper-cut, Kidney punch, Timing, Footwork, Accidental Butt, Bleeder, Bolo Punch, Bout, Brawler, Break, Buckle, Canvas, Card, Caught Cold, Clinch, Corkscrew Punch, Cornerman, Counterpunch, Cross, Cutman, Dive, Eight Count, Glass Jaw, Haymaker, Liver Shot, Low Blow, Mauler, Neutral Corner, Plodder, Ring Generalship, Roughhousing, Southpaw, Spar, Stablemate, Technical Knockout, Walkout Bout, Whiskers,Pinching, Homerun, Base runner, Perfect game, Throw, Strike, Put out,Free throw, Common foul, Underhead, Technical foul,  Overhead,Relay, Track, Lane, Photo Finish, Hurdles, Shot Put, Discuss Throw, Hammer Throw, High Jump, Triple Jump Cross Country, Gambit, move, resign, stalemate, checkmate, Grandmaster,  international master, Kings Indian Defence".split(','))
sp= [x.strip().lower() for x in sp]

cli= list('''culture
climate
situation
environmental
surroundings
ecology
ecosystem
setting
nature
ambiance
ecosystems
ecological
context
atmosphere
habitat
surround
milieu
landscape
sustainable
ambience
environ
environs
world
ecologically
economy
health
environmentally responsible
climates
environmental sustainability
weather
society
climatic conditions
infrastructure
people
marine ecosystem
environment
community
planet
wildlife habitats
economic
system
sustainably
landscapes
atmospheres
soil
home
language
agriculture
medicine
environmentalism
oasis
impacts
stewardship
nurturing
weather
warming
environment
temperature
precipitation
global warming
environmental
humidity
climate change
greenhouse gas
atmospheric
biodiversity
meteorology
latitude
clime
climatic
rainfall
köppen climate classification
biosphere
oceans
economic
pollution
global
arctic
conditions
change
energy
atmospheric pressure
landscape
economy
tropical
wind
ecology
climate
condition
situation
greenhouse gases
temperate
greenhouse gas emissions
biome
paleoclimatology
monsoon
geography
drought
anthropogenic
location
future
zastruga
ocean current
culture
soil
carbon emissions
ecological
changing
collapse
climatologists
carbon emission
water
reducing carbon emissions
terrain
permafrost
greenhouse gasses
area
earth
hydrosphere
atmospheric carbon dioxide
resource
greenhouse gas emission
science
social
season
co2
region
world meteorological organization
war
infrastructure
subarctic
green
tropical forests
marine ecosystems
warmer temperatures
pandemic
greenhouse emissions
sustainability
nature
carbon
natural
country
carbon dioxide emissions
radio
carbon sequestration
climatologist
climatic conditions
ambient
emission
temp
ocean acidification
emissions
world
geographic
polar regions
planet
covid
copenhagen
air
state
ecosystems
heat trapping gases
overpopulation
ecosystem
habitat
zone
political
climate change ipcc
the
animal
earth's atmosphere
australia
health
co2 emissions
storm
eco
volume
continent
geoengineering
system
elevation
infotainment
housing
inflation
thunderstorm
growing
time
healthcare
disasters
economic downturn
crisis
issues
carbon dioxide
melting glaciers
room
land
enviroment
disaster
climes
reduce carbon emissions
regulatory frameworks
cultural
politics
polar ice
temperatures
climate classification
macro economic
seasons
population
audio
societal
environmentalism
capitalism
nuclear
geology
media
crop yields
humid weather
snowfall
tropical forest
topography
temps
'''.split())

ent= list("""film
television
cinema
fun
funny
cheers
filmmaking
shows
beer
wine
vodka
soundtrack
picture
movie projector
dvd
screenplay
celluloid
film noir
movie theater
studio
sequel
telefilm
flick
series
actor
hero
heroine
srk
shah rukh khan
thriller
director
episode
filmography
box office
film stock
animation
reshoot
show
documentary
genre
videotape
video
motion picture
scene
home movie
kinetoscope
cinematography
documentary film
sitcom
feature film
photographic film
stories
episodes
movie screen
specials
seasons
hollywood
actors
slow motion
sound
photography
comedies
silent movie
movie
theater
hollywood blockbusters
story
music
picture show
moving picture
romantic comedies
romcom
friends
mondler
movieland
comics
books
cameraman
skin flick
characters
read
superman returns
blockbuster
movie star
news
lionsgate
trailers
culture
transformers
cartoon
theaters
collage film
filmmakers
editing
talkies
helmer
imdb
sound film
games
movie camera
feature
musical
final cut
filmmaker
studio system
pic
flicks
hindi films
entertainment
dvds
video camera
sequels
videos
documentaries
remakes
moviedom
shot
bollywood
talkie
soundtracks
script
serials
movie buffs
dramas
media
art
dramedy
cinemas
silents
cartoons
prequels
videogames
epics
computer-generated imagery
novels
sitcoms
horror flick
films
hollywood studios
speed racer
remake
horror
songs
gaming
sci fi
oater
incredibles
thrillers
playing
phi phenomenon
pulp fiction
ovas
theatre
book
arthouse
miniseries
short subject
franchises
animes
tvs
dergarabedian
netflix
shorts
preview
sports
roles
zoetrope
reading
game
plays
drama
propaganda film
tvshows
porn
graphic novels
content
citizen kane
grindhouse
pop
film frame
streaming
directors
movie theatre
persistence of vision
photos
docs
web
moviemakers
roger corman
listen
multiplexes
images
etc
dinner
die hard
musicals
audience
listening
concerts
escapist
literature
short
performances
silver screen
screenwriter
animations
soap operas
comic
shoes
commercials
film score
scripts
batman begins
live
ova
disney
played
programs
trilogies
ads
events
magic lantern
quentin tarantino
clips
clone
cult classic
pics
genres
toys
technicolor
bands
toy story 3
moviemaker
albums
streams
mcu
webseries
science fiction
widescreen
podcasts
avatar
auguste and louis lumière
animated
out
kdramas
mini
star wars
soderbergh
schlock
character
have
vids
franchise
social
scary movie
movielink
comedy
horror genre
week
universes
watch
marvel
manga
moviegoing
productions
romantic comedy
matrix reloaded
years
true
whatever
ideas
food
webisodes
and
eat
scorsese
communication
prequel
theme
star
filler
year
vudu
moviegoer
movement
projectionist
subtitles
novel
dubbing
translation
3d
take
caption
sequence
infotainment
credit
synchronize
dub
movieverse
production
docudrama
batman
credits
tape
product
subtitle
costumes
dances
audiences
filmy
storyboards
suspenseful
photochemistry
andrei tarkovsky
sound-on-film
microfilm
bioscope
cinematograph
minimovie
pix
pellicle
midmovie
movielike
projector
comic book
reel
matinee
videocassette
preproduction
three-d
synchronise
3-d
shoot-'em-up
cinemagoing
filmzine
metafilm
nanofilm
filmize
terminator
theatrical
photoplay
moving-picture show
motion-picture show
polaroid
filmdom
parody
filmlike
organ
piano
tv
spoof
orchestra
phenakistoscope
mockumentary
reality
broadway
praxinoscope
starring
hbo
popcorn
best
filmology
starred
fantasy
filmstrip
featured
acclaimed
film theory
adaptation
directed
filmed
stars
fun
screen
love
zoopraxiscope
filmgoing
presents
monster
funny
vcr
tale
cast
ever
fandub
wonder
cameo
adventure
like
featuring
titled
crazy
imagine
goes
romantic
premiere
directing
gets
joke
eastwood
stranger
art film
version
entertain
film industry
showmance
titanic
vidcap
visual effects
extravaganza
entertainer
digital intermediate
digital recording
tv movie
adult movie
teleplay
cultural artifact
move picture
science fiction movie
burlesque
semiotics
filmwise
showdown
scriptment
blockbusters
shoot em up
wittgenstein
cinemuck
autoradiograph
zombie
language
suspense
popcorn movie
producer
hatedom
showbusiness
camcording
kinescope
undercranked
westerns
recording medium
undercrank
backdrop
trekiverse
cinema verite
internet
musical theater
coming attraction
rough cut
talking picture
silent picture
moviegoers
musical comedy
filmcoated
united states
counterpoint
screenwork
set construction
sticky floor
overexpose
scriptwriting
exhibitor
opera
teleshow
ballet
newspaper
row of seat
pageantry
shrink wrap
mise en scène
magazine
filmable
show movie
movie theatre space
mumblecore
fun to watch
cable television
freeze frame
showie
preshow
closeup
photographic emulsion
compere
telerecording
photofinisher
celebrity
eadweard muybridge
photographic plate
forthshow
showable
supershow
telescreen
baby wrangler
showlike
mumbai
stalkumentary
chick flick
unfilmed
film at 11
at theater
hindi
downstage
woodville latham
star trek
sword and sorcery
koster and bial's music hall
tittytainment
big picture
new york city
horror movie
film editing
animatronics
lobby card
edutainment
visual arts
loudspeaker
silent film
voice actor
theater box
on television
d. w. griffith
cinema of the united states
drive in movie
artistic
theoretical
action
home video
commerce
technology
theater ticket
friedrich wilhelm murnau
fritz lang
charles chaplin
fade to black
buster keaton
balcony seat
media maven
multiple exposure
sound effects
plot
trilogy
capote
cineplex
beowulf
cinemascope
plotline
peckinpah
featurette
kubrick
directorial
auteur
color motion picture film
double exposure
short film
anti fandom
view audience
film set
india
moviemaking
variety show
becky sharp
waterworld
horse opera
stereophonic sound
polarized 3d system
animate cartoon
peter bogdanovich
martin scorsese
movies
anime
comic books
screenplays
youtube
play
fiction
sound recording and reproduction
scenes
dialogue
the
other
pictures
projects
restaurants
cinematic
people
acting
spy kids
things
mall
any
new
dvd rentals
eating
products
magazines
sergei eisenstein
advertising
shoot
exposure
filmic
intolerance
filming
trekverse
superscreen
christiaan huygens
movie ticket
overexposure
the birth of a nation
hearing
biopic""".split())

sample=df_insta_posts.sample(withReplacement=False, fraction=0.0001, seed=42)

detect_language_udf = F.udf(detect_language, StringType())
categorization_dict_udf= F.udf(categorization_dict, StringType())

sample_lang= sample.withColumn('lang', detect_language_udf(sample['description']))
# filtering and taking only the english posts for categorization -> took 16 mins to run for sample (20% of full) data
en_df = sample_lang.filter(sample_lang["lang"] == "en")

# cleaning the description feature
regexTokenizer = RegexTokenizer(inputCol="description", outputCol="words", pattern="\\W")

#removing stop words
add_stopwords = ["http","https","amp","rt","t","c","the","0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"] 
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(en_df)
dataset = pipelineFit.transform(en_df)

dataset_category= dataset.withColumn('Category', categorization_dict_udf(F.col('filtered')))
dataset_category.show(5, False)

dataset_category.groupby('Category').count().show()

+--------+-----------+-----------+----------+---------------+-----------------------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+---------------+----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------+
|sid     |sid_

In [ ]:
#-----------------------------------------------------------------------------
# Instagram :: Does ‘#likes’+’#comments’ depend on ‘post-type’? Eg. Do videos have more chances of getting comments/ photos have more chances of getting likes. 
# Follow-up-> What combination of photos/videos is likely to get more likes/comments.
#-----------------------------------------------------------------------------

#Joining posts and profiles instagram dataset
# sample_profile= df_insta_profiles.sample(withReplacement=False, fraction=0.01, seed=42)
join_profile_posts= df_insta_profiles.join(dataset_category, df_insta_profiles.profile_id==dataset_category.profile_id, "inner")

#Filtering the business accounts
join_df= join_profile_posts.filter(join_profile_posts['is_business_account']=='true')
join_df.groupBy('Category').count().orderBy('count').show(20, False)


# 5 Most popular business account
df_spark= join_df.withColumn('Popularity_index', (((10*join_profile_posts['followers'])-join_profile_posts['following'])/(0.1* join_profile_posts['n_posts'])))

df_popular=df_spark.sort(F.col('Popularity_index').desc())
df_popular.select('profile_name','is_business_account').show(5)

+-------------------------+-----+
|Category                 |count|
+-------------------------+-----+
|Sports                   |7    |
|Environment&Climate      |60   |
|Politics,Business&Finance|209  |
|Entertainment            |235  |
+-------------------------+-----+

+------------------+-------------------+
|      profile_name|is_business_account|
+------------------+-------------------+
|    smoothiethecat|               true|
|      rosa_fidilio|               true|
|            _sovya|               true|
|rohit_khandelwal77|               true|
|        anakarylle|               true|
+------------------+-------------------+
only showing top 5 rows



In [ ]:
#-----------------------------------------------------------------------------
# Instagram :: Given that it is a business_account what are the categories of posts? 
#-----------------------------------------------------------------------------

#Defining Likability= (numbr_likes+ 10* number_comments) and comparing with post_type
#post_type-> 1 - Photo, 2 - Video, 3 - multy

df_likability= df_insta_posts.withColumn('likability', df_insta_posts['numbr_likes']+ (10* df_insta_posts['number_comments']))

#Removing gibberish values for post_type feature
df_likability=df_likability.withColumn('post_type', F.when((df_likability['post_type']=='1') | \
                                (df_likability['post_type']=='2') | \
                                (df_likability['post_type']=='3'), df_likability['post_type'] )\
                                .otherwise(None))
df_likability = df_likability.dropna()
df_likability = df_likability.groupby('post_type').sum('likability')
df_likability.select("post_type", format_number(col("sum(likability)"),0).alias("likability")).show()

+---------+-------------+
|post_type|   likability|
+---------+-------------+
|        3|       23,211|
|        1|9,527,306,750|
|        2|  558,286,809|
+---------+-------------+

